# Subway_Timetable_DB

http://subway.koreatriptips.com/subway-station.html 에서 지하철 역당 시간표를 Crawling 한다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [2]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

---

지하철 호선과 역당 시간을 저장한다. 

- function name : for_subway_timetable

In [3]:
def for_subway_timetable():
    url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%A7%80%ED%95%98%EC%B2%A0+%EC%8B%9C%EA%B0%84%ED%91%9C"

    # chrome webdriver 이용
    browser = webdriver.Chrome('/Users/sony/Downloads/chromedriver_win32/chromedriver')
    browser.get(url)
    time.sleep(1)

    html = browser.page_source
    dom = BeautifulSoup(html, "lxml")
    browser.find_element_by_css_selector("a.select._lane_selectbox").click() # line 옵션 선택 버튼 클릭
    for_line = dom.select("div.traff_slayer._lane_layer li > a") # line links 받음

    conn = engine.connect()

    for_pass = [12, 14, 17, 18, 19]
    for i in range(1, 21):
        if i not in for_pass:
            if i != 1:
                browser.find_element_by_css_selector("a.select._lane_selectbox").click() # line 옵션 선택 버튼 클릭
            href = "div.traff_slayer._lane_layer li:nth-child(" + str(i+1) + ") > a"
            browser.find_element_by_css_selector(href).click()

            html = browser.page_source
            dom = BeautifulSoup(html, "lxml")
            for_station = dom.select("div.traff_slayer._station_layer li > a") # station links 받음

            for j in range(1, len(for_station)):
                if j != 1:
                    browser.find_element_by_css_selector("a.select._lane_selectbox").click()
                    href = "div.traff_slayer._lane_layer li:nth-child(" + str(i+1) + ") > a"
                    browser.find_element_by_css_selector(href).click()
                href = "div.traff_slayer._station_layer li:nth-child(" + str(j+1) + ") > a"
                browser.find_element_by_css_selector(href).click()

                browser.find_element_by_css_selector("div.traf_section._subway_schedule_content a.search").click() # 검색 버튼 클릭

                browser.switch_to.window(browser.window_handles[1]) # 클릭으로 생성된 새 탭으로 이동
                time.sleep(2)
                browser.find_element_by_css_selector("#_subwayTimetable > a").click() # 열차 시간표 클릭
                html = browser.page_source
                dom = BeautifulSoup(html, "lxml")

                for_time = dom.select("#_dayTabContents tbody tr") # 시간표 받아옴

                for_week_count = 0
                for_day_of_week = ['weekday', 'Saturday', 'Sunday/holiday']
                for k in range(1, len(for_time)):
                    if for_time[k].th.text == '종착':
                        for_week_count += 1
                    else:
                        for_timetable = {}  # line/station/hour/minute
                        for_timetable['line'] = for_line[i].text # line 이름 저장
                        for_timetable['station'] = for_station[j].text # station 이름 저장
                        for_timetable['day_of_week'] = for_day_of_week[for_week_count] # 요일 저장 - 평일/ 토요일/ 일요일,공휴일
                        for_timetable['hour'] = int((for_time[k].th.text)[:-1]) # 시간 저장
                        
                        for direc in range(2):
                            for_timetable['direction'] = for_time[k].find_all('td')[direc]['class'][0] # 상행인지 하행인지 저장
                            for for_minute in for_time[k].find_all('td')[direc].find_all('li'):
                                if for_minute.text != '-':
                                    for_timetable['minute'] = int(for_minute.text) # 분 저장
                                    for_timetable['last_station'] = for_minute['data-arrivalstation']
                                    conn.execute(subway_timetable.insert(), for_timetable) # DB에 저장
                browser.close()
                browser.switch_to.window(browser.window_handles[0]) # 원래의 탭으로 이동
                time.sleep(2)

---

subway_timetable.db 정의

In [4]:
engine = create_engine("sqlite:///subway_timetable.db", echo = True)
metadata = MetaData()

subway_timetable = Table('subway_timetable', metadata,
                        Column('id', Integer, primary_key =True),
                        Column('line', String, nullable = False),
                        Column('station', String, nullable = False),
                        Column('direction', String, nullable = False),
                        Column('day_of_week', String, nullable = False), 
                        Column('last_station', String), 
                        Column('hour', Integer),
                        Column('minute', Integer)
                    )

metadata.create_all(engine)

2018-08-18 15:02:39,700 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-18 15:02:39,703 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 15:02:39,705 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-18 15:02:39,706 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 15:02:39,708 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("subway_timetable")
2018-08-18 15:02:39,709 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 15:02:39,712 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE subway_timetable (
	id INTEGER NOT NULL, 
	line VARCHAR NOT NULL, 
	station VARCHAR NOT NULL, 
	direction VARCHAR NOT NULL, 
	day_of_week VARCHAR NOT NULL, 
	last_station VARCHAR, 
	hour INTEGER, 
	minute INTEGER, 
	PRIMARY KEY (id)
)


2018-08-18 15:02:39,713 INFO sqlalchemy.engine.base.Engine ()
2018-08-18 15:02:39,754 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
for_subway_timetable()

---